In [88]:
import torch
import os
import argparse
import pandas as pd

In [35]:
from module.model import model as tagger
from module.model import device
from module.model import tokenizer
from module.model import id2label

In [89]:
parser = argparse.ArgumentParser()
parser.add_argument("-q", "--query", dest="keyword", action="store")
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [-q KEYWORD]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"9b745e73-fdfa-47a4-a3d7-2e5b936047b6" --shell=9002 --transport="tcp" --iopub=9004 --f=/home/ubuntu/.local/share/jupyter/runtime/kernel-v2-17239VsodKbOR2SWS.json


SystemExit: 2

/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Loading

In [36]:
file_list = os.listdir('./inference_data')
file_path = []
for file_name in file_list:
    tmp_path = os.path.join('./inference_data', file_name)
    file_path.append(tmp_path)
file_path.sort()

In [37]:
load_df = lambda x: pd.read_csv(x)
dfs = [load_df(x) for x in file_path]
data = dfs[0]
data.dropna(inplace=True)
# data[pd.isna(data).any(axis=1)]
# data.isna().nunique()
print(f'length of {file_path[0]}: {len(data)}')

length of ./inference_data/event_port.csv: 171


In [38]:
data.isnull().any()

Event    False
dtype: bool

## Cleansing

In [39]:
# def preprocess(df):
#     df['Event'] = df['Event'].str.replace("．", ".", regex=False)
#     df['Event'] = df['Event'].astype(str)
#     # df['Event'] = df['Event'].str.replace(r'[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', " ", regex=True)
#     return df

In [40]:
# data = preprocess(data)

In [86]:
keyword = '화물'

data['Indexes'] = data.Event.str.find(keyword)
infer_data = data[data.Indexes > -1]
# infer_data

In [85]:
# for event in infer_data.Event:
#     print(event)
#     print()

In [68]:
infer_data_lst = infer_data.Event.apply(lambda x: tokenizer(x, max_length=256, padding='max_length', truncation=True))
input_ids_lst = []
input_masks_lst = []
for e in infer_data_lst:
    input_ids_lst.append(e['input_ids'])
    input_masks_lst.append(e['attention_mask'])
len(input_ids_lst), len(input_masks_lst)

(36, 36)

In [69]:
def infer(input_ids_lst, input_masks_lst):

    tagger.eval()
    with torch.no_grad():
        infer_results = []
        for input_ids, input_masks in zip(input_ids_lst, input_masks_lst): 
            input_tensor = torch.LongTensor(input_ids).unsqueeze(0).to(device)
            mask_tensor = torch.LongTensor(input_masks).unsqueeze(0).to(device)
            output = tagger(input_tensor, mask_tensor)
            pred = torch.argmax(output, dim=-1).squeeze().detach().cpu().tolist()
            infer_results.append(pred)

        return infer_results
    
infer_results = infer(input_ids_lst, input_masks_lst)

In [70]:
subwordsList = []
tagsList = []
toLabel = lambda x: id2label[x]
for input_ids, result in zip(input_ids_lst, infer_results):
    decoded = tokenizer.decode(input_ids, skip_special_tokens=True)
    subwords = tokenizer.encode(decoded)[1:-1]
    subwords = list(map(lambda x: tokenizer.convert_ids_to_tokens(x), subwords))
    subwordsList.append(subwords)
    lenSubwords = len(subwords)    

    result = result[1:lenSubwords+1]
    tags = []
    for id in result:
        tags.append(toLabel(id))
    tagsList.append(tags)

In [78]:
# 날짜(DAT), 시간(TIME), 장소(LOC), 작업 정보(WRK)를 추출해서 txt 파일에 저장

def extract(tagtype, tags, subwords):
    # if tag does not exists then notify and pass
    # if not '{tag}_I' then stop
    tag_start = f'{tagtype}_B'
    if tag_start in tags:
        b_idx = tags.index(tag_start)
        for i, tag in enumerate(tags[b_idx:]):
            if i > 0 and tag != f'{tagtype}_I':
                # print(tags[b_idx:b_idx+i+1])
                # print(tags[b_idx:b_idx+i])
                tagged = subwords[b_idx:b_idx+i]
                tagged = tokenizer.convert_tokens_to_string(tagged)
                # print(tagged)
                return tagged
    else:
        msg = f'{tagtype} not found.'
        # print(msg)
        return msg

tagtypes = dict(
    dat='DAT',
    tim='TIM',
    loc='LOC',
    wrk='WRK',    
)

# subwordsList, tagsList
nerResults = []
count = 0
for subwords, tags in zip(subwordsList, tagsList):
    # print(f'index: {count}')
    extracted = []
    for tagtype in tagtypes.values():
        # print(tagtype)
        tagged = extract(tagtype, tags, subwords)
        extracted.append(tagged)
        # print()
    nerResults.append(extracted)
    count += 1

In [80]:
# for i in nerResults:
#     print(i)

In [76]:
df = pd.DataFrame(nerResults, columns=['Date', 'Time', 'Location', 'Work'])

In [84]:
df['Date'], df['Time'], df['Location'], df['Work'] = df['Date']+'\t', '\t'+df['Time']+'\t', '\t'+df['Location']+'\t', '\t'+df['Work']
df.to_csv('./inference_result.txt', index=False, sep='|')